<a href="https://colab.research.google.com/github/prgawade/battlefieldofAI/blob/main/S4/S4-Assignment-Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from __future__ import print_function
#PyTorch is an open source machine learning framework
#PyTorch is a Python package that provides two high-level features:
#Tensor computation (like NumPy) with strong GPU acceleration
#Deep neural networks built on a tape-based autograd system
import torch
# A neural networks library deeply integrated with autograd designed for maximum flexibility
import torch.nn as nn
#Convolution functions such as conv1d , conv2d, conv3d
import torch.nn.functional as F
#torch.optim is a package implementing various optimization algorithms.
# Adam - Implements Adam algorithm
# SGD Implements stochastic gradient descent (optionally with momentum).
import torch.optim as optim
# The torchvision package consists of popular datasets, model architectures, and common image transformations for computer vision.
# Transforms are common image transformations available in the torchvision.transforms module. 
from torchvision import datasets, transforms

In [30]:
from torch.nn.modules.dropout import Dropout2d
import torch.nn as nn
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=128,
        kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=128)
        self.tns1 = nn.Conv2d(in_channels=128, out_channels=8,
        kernel_size=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16,
        kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=16)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=16,
        kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=16)
        self.conv4 = nn.Conv2d(in_channels=16, out_channels=32,
        kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=32)
       
        self.pool2 = nn.MaxPool2d(2, 2)
        self.tns2 = nn.Conv2d(in_channels=32, out_channels=16,
        kernel_size=1, padding=1)
        
        self.conv5 = nn.Conv2d(in_channels=16, out_channels=16,
        kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(num_features=16)
        self.conv6 = nn.Conv2d(in_channels=16, out_channels=32,
        kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(num_features=32)
       # self.conv7 = nn.Conv2d(in_channels=32, out_channels=32,
        #kernel_size=1, padding=1)
        self.gpool = nn.AvgPool2d(kernel_size=9)

        self.out1 = nn.Sequential(
            nn.Linear(32, 10)
        )
        self.drop = nn.Dropout2d(0.1)
    def forward(self, x):
        x = self.tns1(self.drop(self.bn1(F.relu(self.conv1(x)))))
        x = self.drop(self.bn2(F.relu(self.conv2(x))))
        x = self.pool1(x)
        x = self.drop(self.bn3(F.relu(self.conv3(x))))
        x = self.drop(self.bn4(F.relu(self.conv4(x))))
        x = self.tns2(self.pool2(x))
        x = self.drop(self.bn5(F.relu(self.conv5(x))))
        x = self.drop(self.bn6(F.relu(self.conv6(x))))
        #print(x.shape)
        #x = self.conv7(x)
        #print(x.shape)
        x = self.gpool(x)
        x = x.view(-1, 32)
        x = self.out1(x)
        #print(x.shape)

        return F.log_softmax(x)

In [31]:
!pip install torchsummary
#API to view the visualization of the model, which is helpful while debugging your network.
from torchsummary import summary
# Returns a bool indicating if CUDA is currently available.
# NVIDIA’s CUDA is a general purpose parallel computing platform and programming model that accelerates deep learning and other compute-intensive apps by taking advantage of the parallel processing power of GPUs.
use_cuda = torch.cuda.is_available()
# A torch.device is an object representing the device on which a torch.Tensor is or will be allocated.
device = torch.device("cuda" if use_cuda else "cpu")
# Specify which device to use
model = Net().to(device)
# Model summary
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 28, 28]           1,280
       BatchNorm2d-2          [-1, 128, 28, 28]             256
         Dropout2d-3          [-1, 128, 28, 28]               0
            Conv2d-4            [-1, 8, 30, 30]           1,032
            Conv2d-5           [-1, 16, 30, 30]           1,168
       BatchNorm2d-6           [-1, 16, 30, 30]              32
         Dropout2d-7           [-1, 16, 30, 30]               0
         MaxPool2d-8           [-1, 16, 15, 15]               0
            Conv2d-9           [-1, 16, 15, 15]           2,320
      BatchNorm2d-10           [-1, 16, 15, 15]              32
        Dropout2d-11           [-1, 16, 15, 15]               0
           Conv2d-12           [-1, 32, 15, 15]           4,640
    

<ipython-input-30-2e48604e0afa>:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [32]:
from tqdm import tqdm
# model defined above
# device in this case will cpu
# train loader to download training images and transform into tensor 
# SGD optimizer
# The learning rate controls how quickly the model is adapted to the problem. Smaller learning rates require more training epochs given the smaller changes made to the weights each update, whereas larger learning rates result in rapid changes and require fewer training epochs.
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # Sets the gradients of all optimized torch.Tensor s to zero.
        output = model(data)
        # loss function
        loss = F.nll_loss(output, target) # loss function - The negative log likelihood loss.
        loss.backward() 
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [33]:

#Sets the seed for generating random numbers. Returns a torch.Generator object.

torch.manual_seed(1)
#batch_size = 128
batch_size = 128
#PyTorch's DataLoader class, which in addition to our Dataset class, also takes in the following important arguments:

#batch_size, which denotes the number of samples contained in each generated batch.
#shuffle. If set to True, we will get a new order of exploration at each pass (or just keep a linear exploration scheme otherwise). Shuffling the order in which examples are fed to the classifier is helpful so that batches between epochs do not look alike. Doing so will eventually make our model more robust.
#num_workers, which denotes the number of processes that generate batches in parallel. A high enough number of workers assures that CPU computations are efficiently managed, i.e. that the bottleneck is indeed the neural network's forward and backward operations on the GPU (and not data generation).
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
# Download training data 
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, 
                    transform=transforms.Compose([ # Composes several transforms together.
                        transforms.ToTensor(), # Convert a PIL Image or numpy.ndarray to tensor.
                        transforms.Normalize((0.1307,), (0.3081,)) # Normalize(mean, std[, inplace]) Normalize a tensor image with mean and standard deviation.
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [34]:

model = Net().to(device)

#torch.optim.SGD(params, lr=<required parameter>, momentum=0, dampening=0, weight_decay=0, nesterov=False, *, maximize=False, foreach=None, differentiable=False)
#params (iterable) – iterable of parameters to optimize or dicts defining parameter groups

#lr (float) – learning rate

#momentum (float, optional) – momentum factor (default: 0)

#weight_decay (float, optional) – weight decay (L2 penalty) (default: 0)

#dampening (float, optional) – dampening for momentum (default: 0)

#nesterov (bool, optional) – enables Nesterov momentum (default: False)

#maximize (bool, optional) – maximize the params based on the objective, instead of minimizing (default: False)

#foreach (bool, optional) – whether foreach implementation of optimizer is used (default: None)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
epochs = 20
for epoch in range(1, epochs + 1):
    print('epoch value is , ', epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

# Test set: Average loss: 0.3189, Accuracy: 8758/10000 (88%) - lr=0.01, momentum=0.9 , epoch 1, batch_size = 128
# Test set: Average loss: 0.4987, Accuracy: 7927/10000 (79%) - lr=0.02, momentum=0.9 , epoch 1 , batch_size = 128
# Test set: Average loss: 1.8482, Accuracy: 3000/10000 (30%) - lr=0.03, momentum=0.9 , epoch 1 , batch_size = 128
# lr=0.01, momentum=0.9 , epoch 9
# accuracy at epoch 3
#Test set: Average loss: 0.0377, Accuracy: 9874/10000 (99%)

# accuracy starts decreasing for same parameters even after run time restart
#Test set: Average loss: 0.7211, Accuracy: 6943/10000 (69%) # lr=0.01, momentum=0.9 , epoch 4  , batch_size = 128

### changing batch size to 16

#Test set: Average loss: 1.4329, Accuracy: 4815/10000 (48%) # lr=0.01, momentum=0.9 , epoch 4  , batch_size = 16


epoch value is ,  1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-30-2e48604e0afa>:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.267739862203598 batch_id=468: 100%|██████████| 469/469 [00:23<00:00, 20.10it/s]



Test set: Average loss: 0.1443, Accuracy: 9631/10000 (96%)

epoch value is ,  2


loss=0.19418276846408844 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.23it/s]



Test set: Average loss: 0.0666, Accuracy: 9803/10000 (98%)

epoch value is ,  3


...


Test set: Average loss: 0.0203, Accuracy: 9922/10000 (99%)

epoch value is ,  19


loss=0.03545418009161949 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.84it/s]



Test set: Average loss: 0.0203, Accuracy: 9932/10000 (99%)

epoch value is ,  20


loss=0.024912239983677864 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.94it/s]



Test set: Average loss: 0.0192, Accuracy: 9942/10000 (99%)

